In [1]:
import pandas as pd

In [ ]:
# df = df[df['Best Entity literalString'] != 'Type']
# df = df[df['Best Entity literalString'] != 'FORUMS']

In [2]:
df = pd.read_csv('nell-disease.csv', delimiter='\t', nrows=52556)

In [4]:
len(df)

18630

In [18]:
df = df.drop_duplicates('Entity')

In [19]:
len(df)

17257

In [30]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py

Installed autotime.py. To use it, type:
  %load_ext autotime


In [31]:
%load_ext autotime

In [35]:
df.head(1)

,Entity,Relation,Value,Iteration of Promotion,Probability,Source,Entity literalStrings,Value literalStrings,Best Entity literalString,Best Value literalString,Categories for Entity,Categories for Value,Candidate Source,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,psychogenic_pruritus,concept:haswikipediaurl,http://en.wikipedia.org/wiki/Psychogenic%20pru...,710,0.95,MBL-Iter%3A710-2013%2F03%2F13-12%3A17%3A37-Fro...,Psychogenic pruritus,http://en.wikipedia.org/wiki/Psychogenic%20pru...,Psychogenic pruritus,http://en.wikipedia.org/wiki/Psychogenic%20pru...,NaN,NaN,%5BAliasMatcher-Iter%3A621-2012%2F08%2F03-10%3...,NaN,NaN,NaN,NaN


time: 18.5 ms


In [54]:
df = df.drop_duplicates('Value')
df = df[pd.notnull(df['Value'])]

time: 4.36 ms


In [55]:
df.Value.describe()

count                                                1805
unique                                               1805
top       http://en.wikipedia.org/wiki/Cirsoid%20aneurysm
freq                                                    1
Name: Value, dtype: object

time: 6.84 ms


In [58]:
import urllib
import urllib2
from BeautifulSoup import BeautifulSoup
import re

lst = []

def clean(text):
    return html.fromstring(text).text_content().lower().strip()

for i, row in enumerate(df.iterrows()):
    opener = urllib2.build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/5.0')] #wikipedia needs this
    
    
    try:
        resource = opener.open(row[1]['Value'])
    except ValueError:
        lst.append(None)
        continue
    except urllib2.HTTPError:
        lst.append(None)
        continue
    except urllib2.URLError:
        lst.append(None)
        continue

    data = resource.read()
    resource.close()
    soup = BeautifulSoup(data)
    lst.append(re.sub('<[^<]+?>', '', str(soup.find('div',id="bodyContent").p)))

time: 13min 51s


In [59]:
df['wiki'] = lst

time: 35.8 ms


In [60]:
df2 = df[['Best Entity literalString', 'wiki', 'Entity']]

time: 2.81 ms


In [65]:
df2 = df2.drop_duplicates('wiki')
df2 = df2[df2.wiki != None]

time: 3.45 ms


In [66]:
df2.describe()

,Best Entity literalString,wiki,Entity
count,1746,1745,1746
unique,1746,1745,1746
top,Johanson Blizzard syndrome,3-Methylcrotonyl-CoA carboxylase deficiency (3...,jean_marc_gaspard_itard
freq,1,1,1


time: 16.3 ms


In [68]:
df2.head()

,Best Entity literalString,wiki,Entity
0,Psychogenic pruritus,Psychogenic pruritus is a common manifestation...,psychogenic_pruritus
1,Hydrophthalmos,Hydrophthalmos is a congenital form of glaucoma.,hydrophthalmos
2,Amino acid transport disorder,Amino acid transport disorders are medical con...,amino_acid_transport_disorder
3,Adelaide Hanscom,Adelaide Hanscom Leeson (25 November 1875 – 19...,adelaide_hanscom
4,trachoma,"Trachoma, also called granular conjunctivitis,...",trachoma


time: 8.53 ms


In [69]:
df2.to_csv('full_output.csv', index=False)

time: 33.7 ms


In [73]:
df2.columns[0]

'Best Entity literalString'

time: 1.94 ms


In [76]:
df2 = df2.drop_duplicates(df2.columns[0])
df2 = df2.drop_duplicates(df2.columns[1])
df2 = df2.drop_duplicates(df2.columns[2])

time: 5.39 ms


In [77]:
len(df)

1805

time: 1.91 ms


In [79]:
import numpy as np

time: 841 µs


In [81]:
np.zeros(1746)

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

time: 2.37 ms


In [80]:
len(df2)

1746

time: 2.03 ms


In [86]:
df2['cls'] = np.zeros(1746)

time: 1.74 ms


In [87]:
df2.head()

,Best Entity literalString,wiki,Entity,cls
0,Psychogenic pruritus,Psychogenic pruritus is a common manifestation...,psychogenic_pruritus,0
1,Hydrophthalmos,Hydrophthalmos is a congenital form of glaucoma.,hydrophthalmos,0
2,Amino acid transport disorder,Amino acid transport disorders are medical con...,amino_acid_transport_disorder,0
3,Adelaide Hanscom,Adelaide Hanscom Leeson (25 November 1875 – 19...,adelaide_hanscom,0
4,trachoma,"Trachoma, also called granular conjunctivitis,...",trachoma,0


time: 10.5 ms


In [88]:
df2.to_csv('full_output2.csv', index=False)

time: 15.4 ms
